<h1>Introducción a LanceDB<h1>

<img src='../images/lancedb-logo.svg' width=600>

LanceDB es una base de datos vectorial de código abierto para IA diseñada para almacenar, administrar, consultar y recuperar incrustaciones en datos multimodales a gran escala. El núcleo de LanceDB está escrito en Rust 🦀 y está construido sobre Lance , un formato de datos en columnas de código abierto diseñado para cargas de trabajo de ML de alto rendimiento y acceso aleatorio rápido.

La mayoría de las bases de datos vectoriales existentes almacenan y consultan únicamente las incrustaciones y sus metadatos. Los datos reales se almacenan en otro lugar, por lo que es necesario administrar su almacenamiento y control de versiones por separado.

LanceDB admite el almacenamiento de los datos reales , junto con las incrustaciones y los metadatos. Puede conservar sus imágenes, videos, documentos de texto, archivos de audio y más en el formato Lance, que proporciona control automático de versiones de datos y recuperaciones y filtrados ultrarrápidos a través de LanceDB.

<div style="text-align: center;">
    <img src='../images/lancedb_embedded_explanation.png' width="700">
</div>


<h2>Comandos utiles</h2>

Para obtener los nombres de todas las tablas almacenadas: `print("base".table_names())`  
Para abrir una tabla específica: `"variable" = "base".open_table("my_table")`  
Para agregar nuevos elementos a una tabla ya abierta: `"my_table".add(items)`  
Para eliminar elementos de la tabla: `"my_table".delete("nombre_columna" = "valor_especifico")`  
Para eliminar una tabla: `"base".drop_table("my_table")`  
Para gestionar vectores que no cumplen con los requisitos: `"my_table".on_bad_vectors`


In [ ]:
import lancedb
import pandas as pd
import pyarrow as pa
import polars as pl
import numpy as np

: 

<h2>Conexion a la base de datos<h2>

In [2]:
db = lancedb.connect("data/introdb")

<h2>Creación de tablas<h2>

In [3]:
#Desde una lista de tuplas o diccionarios

data = [
    {"vector": [1.1, 1.2], "lat": 45.5, "long": -122.7, "name": "Location A", "category": "Urban", "population": 100000},
    {"vector": [0.2, 1.8], "lat": 40.1, "long": -74.1, "name": "Location B", "category": "Suburban", "population": 50000},
    {"vector": [2.2, 3.4], "lat": 34.9, "long": -118.3, "name": "Location C", "category": "Rural", "population": 2000},
    {"vector": [3.1, 4.5], "lat": 51.5, "long": -0.1, "name": "Location D", "category": "Urban", "population": 120000}
]

table = db.create_table("my_table", data, exist_ok=True, mode="overwrite")

#Desde un DataFrame de Pandas

data = pd.DataFrame({
    "vector": [
        [1.1, 1.2, 1.3, 1.4], 
        [0.2, 1.8, 0.4, 3.6], 
        [3.1, 4.2, 5.3, 6.4],
        [1.9, 2.8, 3.7, 4.6],
        [2.3, 3.5, 4.8, 5.1],
        [0.6, 0.9, 1.1, 1.3]
    ],
    "lat": [45.5, 40.1, 34.9, 51.5, 48.3, 39.7],
    "long": [-122.7, -74.1, -118.3, -0.1, 2.2, -75.5]
})

table_pd = db.create_table("pd_table", data, exist_ok=True, mode="overwrite")


#Desde un DataFrame de Polars

data = pl.DataFrame({
    "vector": [[3.1, 4.1], [5.9, 26.5], [7.2, 9.3], [2.5, 15.8]],
    "item": ["foo", "bar", "baz", "qux"],
    "price": [10.0, 20.0, 15.0, 25.0],
    "quantity": [5, 10, 7, 12],
    "category": ["electronics", "furniture", "books", "clothing"]
})

table_pl = db.create_table("pl_table", data=data, exist_ok=True, mode="overwrite")

print(table_pd.to_pandas())
print()
print(table_pl.to_pandas())
print()
print(table.to_pandas())

                 vector   lat   long
0  [1.1, 1.2, 1.3, 1.4]  45.5 -122.7
1  [0.2, 1.8, 0.4, 3.6]  40.1  -74.1
2  [3.1, 4.2, 5.3, 6.4]  34.9 -118.3
3  [1.9, 2.8, 3.7, 4.6]  51.5   -0.1
4  [2.3, 3.5, 4.8, 5.1]  48.3    2.2
5  [0.6, 0.9, 1.1, 1.3]  39.7  -75.5

        vector item  price  quantity     category
0   [3.1, 4.1]  foo   10.0         5  electronics
1  [5.9, 26.5]  bar   20.0        10    furniture
2   [7.2, 9.3]  baz   15.0         7        books
3  [2.5, 15.8]  qux   25.0        12     clothing

       vector   lat   long        name  category  population
0  [1.1, 1.2]  45.5 -122.7  Location A     Urban      100000
1  [0.2, 1.8]  40.1  -74.1  Location B  Suburban       50000
2  [2.2, 3.4]  34.9 -118.3  Location C     Rural        2000
3  [3.1, 4.5]  51.5   -0.1  Location D     Urban      120000


[2024-12-01T02:57:40Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/introdb/my_table.lance, it will be created
[2024-12-01T02:57:40Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/introdb/pd_table.lance, it will be created
[2024-12-01T02:57:40Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/introdb/pl_table.lance, it will be created


<h2>Tablas vacias</h2>
-Puede crear una tabla vacía para situaciones en las que desee agregar datos a la tabla más adelante. Un ejemplo sería cuando desea recopilar datos de un flujo de datos o un archivo externo y luego agregarlos a una tabla en lotes.  

-Puede utilizar Pydantic para especificar el esquema de la tabla vacía. Tenga en cuenta que no importamos directamente pydantic, sino que utilizamos lancedb.pydantic, que es una subclase de pydantic.BaseModel que se ha ampliado para admitir tipos específicos de LanceDB como Vector. 

In [4]:
from lancedb.pydantic import LanceModel, Vector

class Item(LanceModel):
    vector: Vector(3)  # Vector de 3 dimensiones
    item: str          # Nombre del artículo
    price: float       # Precio del artículo

# Crear la tabla con la nueva estructura
tbl = db.create_table(
    "extended_table", 
    schema=Item.to_arrow_schema(),  # Esquema generado desde el modelo
    exist_ok=True, 
    mode="overwrite"  # Sobrescribe si ya existe
)


[2024-12-01T02:57:45Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/introdb/extended_table.lance, it will be created


<h2>Inserción de datos</h2>

-Puede agregar cualquiera de las estructuras de datos válidas aceptadas por la tabla LanceDB, es decir `dict`, `list[dict]`, `pd.DataFrame`, o `Iterator[pa.RecordBatch]`. A continuación se muestran algunos ejemplos.

In [5]:
# Lista de datos
items = [
    {"vector": [0.1, 0.2, 0.3], "item": "Laptop", "price": 1200.50},
    {"vector": [0.3, 0.1, 0.4], "item": "Headphones", "price": 99.99},
    {"vector": [0.5, 0.2, 0.1], "item": "Smartphone", "price": 799.99}
]

# Insertar datos en la tabla
tbl.add(items)
tbl.to_pandas()


,vector,item,price
0,"[0.1, 0.2, 0.3]",Laptop,1200.50
1,"[0.3, 0.1, 0.4]",Headphones,99.99
2,"[0.5, 0.2, 0.1]",Smartphone,799.99


In [6]:
import random

def make_batches():
    items = ["Mouse", "VideoGame", "Smartphone", "Keyboard", "Monitor"]
    
    for i in range(5): 
        vector = np.random.uniform(0, 10, 3).tolist()  # Valores aleatorios entre 0 y 10
        vector = [round(x, 1) for x in vector]  # Redondear cada valor a 1 decimal
        item = random.choice(items) # Elegir un item aleatorio de la lista
        price = round(random.uniform(5.0, 100.0), 2)  # Precio aleatorio entre 5 y 100
        
        yield [
            {"vector": vector, "item": item, "price": price}
        ]

# Agregar los lotes generados
tbl.add(make_batches())

# Ver los datos generados
tbl.to_pandas()


,vector,item,price
0,"[0.1, 0.2, 0.3]",Laptop,1200.50
1,"[0.3, 0.1, 0.4]",Headphones,99.99
2,"[0.5, 0.2, 0.1]",Smartphone,799.99
3,"[7.7, 3.3, 6.3]",VideoGame,70.25
4,"[1.4, 5.3, 3.6]",Smartphone,53.32
5,"[7.0, 5.0, 9.7]",Keyboard,37.54
6,"[6.4, 7.8, 7.5]",Mouse,56.60
7,"[8.9, 8.5, 8.6]",Mouse,93.88


<h2>Eliminar de una tabla</h2>

Utilice el método `delete()` de las tablas para eliminar filas de una tabla. Para elegir qué filas eliminar, proporcione un filtro que coincida con las columnas de metadatos. Esto puede eliminar cualquier cantidad de filas que coincidan con el filtro.

In [ ]:
tbl.delete('item = "Laptop"')
tbl.to_pandas()

Eliminar de una lista de valores


In [ ]:
to_remove = [99.99,93.89]
to_remove = ", ".join(str(v) for v in to_remove)

tbl.delete(f"price IN ({to_remove})")
tbl.to_pandas()

Eliminar una tabla

In [ ]:
# db.drop_table("extended_table")

<h2>Actualizar una tabla</h2>

Esto se puede utilizar para actualizar de cero a todas las filas según la cantidad de filas que coincidan con la cláusula `where`. Las consultas de actualización siguen la forma de una declaración SQL UPDATE. El parámetro `where` es un filtro SQL que coincide con las columnas de metadatos. Los parámetros `valueso` y `values_sqlse` utilizan para proporcionar los nuevos valores para las columnas.

| Parámetro   | Tipo  | Descripción                                                                                                                 |
|-------------|-------|-----------------------------------------------------------------------------------------------------------------------------|
| where       | str   | La cláusula SQL where que se debe utilizar al actualizar filas. Por ejemplo, 'x = 2' o 'x IN (1, 2, 3)'. El filtro no debe estar vacío o generará un error. |
| values      | dict  | Los valores que se van a actualizar. Las claves son los nombres de las columnas y los valores son los valores que se van a establecer. |
| values_sql  | dict  | Los valores que se van a actualizar. Las claves son los nombres de las columnas y los valores son las expresiones SQL que se van a establecer. Por ejemplo, {'x': 'x + 1'} incrementará el valor de la columna x en 1. |


In [7]:
db = lancedb.connect("data/introdb2")

# Create a table from a pandas DataFrame
data = pd.DataFrame({"x": [1, 2, 3], "vector": [[1, 2], [3, 4], [5, 6]]})
table = db.create_table("my_table", data, exist_ok=True, mode="overwrite")
print(table.to_pandas())

# Update the table where x = 2
table.update(where="x = 2", values={"vector": [10, 10]})
print(table.to_pandas())

   x      vector
0  1  [1.0, 2.0]
1  2  [3.0, 4.0]
2  3  [5.0, 6.0]
   x        vector
0  1    [1.0, 2.0]
1  3    [5.0, 6.0]
2  2  [10.0, 10.0]


[2024-12-01T02:59:41Z WARN  lance::dataset] No existing dataset at /home/jake3120/itam/FuentesDeDatos/LanceDB/notebooks/data/introdb2/my_table.lance, it will be created


<h4>Actualización mediante una consulta SQL</h4>

El parámetro `values` se utiliza para proporcionar los nuevos valores de las columnas como valores literales. También puede utilizar el parámetro `values_sql`/ `valuesSql` para proporcionar expresiones SQL para los nuevos valores. Por ejemplo, puede utilizar values_sql="x + 1"para incrementar el valor de la xcolumna en 1.

In [8]:
# Update the table where x = 2
table.update(values_sql={"x": "x + 1"})

print(table.to_pandas())

   x        vector
0  2    [1.0, 2.0]
1  4    [5.0, 6.0]
2  3  [10.0, 10.0]


<h2>Manejo de vectores erróneos</h2>

En LanceDB Python, puede usar el parametro `on_bad_vectors` para elegir cómo se manejan los valores de vector no válidos. Los vectores no válidos:

1. Son de la dimensión equivocada  
2. Contienen valores NaN  
3. Son nulos pero están en un campo no nulo  

De forma predeterminada, LanceDB generará un error si encuentra un vector incorrecto. También puede elegir una de las siguientes opciones:

- `drop`:Ignorar filas con vectores incorrectos  

- `fill`: Reemplaza los valores incorrectos (NaN) o los valores faltantes (muy pocas dimensiones) con el valor de relleno especificado en el parámetro `fill_value`. Una entrada como [1.0, NaN, 3.0]se reemplazará con [1.0, 0.0, 3.0]if fill_value=0.0. 
 
- `null`: Reemplazar vectores incorrectos por valores nulos (solo funciona si la columna admite valores nulos). Un vector incorrecto [1.0, NaN, 3.0]se reemplazará por valores nulos null si la columna admite valores nulos. Si la columna del vector no admite valores nulos, los vectores incorrectos provocarán un error.